# Notebook to add tremor count to the stations dataset
Using the PNSN tremor catalog and the tremor prep notebook

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
ds = xr.open_dataset("../data/clean_daily_CWU.nc")
ds

# filter tremor days

In [ ]:
tremor = xr.open_dataset('../data/tremor/tremor_2010-2023.nc').sel(time=slice('2010-01-01', '2023-12-31'))
tremor

In [ ]:
ds = ds.sel(time=slice('2010-01-01', '2023-12-31'))
ds

In [ ]:
def get_tremor_count(longitude, latitude):
    delta_dist = 0.5
    tremor_sel = tremor.sel(
                lon=slice(longitude - delta_dist, longitude + delta_dist),
                lat=slice(latitude - delta_dist, latitude + delta_dist)).sum(dim=['lon', 'lat'])[
                'count']
    return tremor_sel

In [ ]:
tremor_dataarray = xr.apply_ufunc(get_tremor_count, ds.longitude, ds.latitude, dask="parallelized", input_core_dims=[[], []], output_core_dims=[['time']], output_dtypes=['float64'], vectorize=True)
tremor_dataarray.compute()

In [ ]:
ds['tremor_count'] = tremor_dataarray
ds

# verrification plots

In [ ]:
ds_notremor = ds.where(ds.tremor_count<2)
ds_notremor['longitude'] = ds_notremor['longitude'].mean(dim='time')
ds_notremor['latitude'] = ds_notremor['latitude'].mean(dim='time')
ds_notremor['height'] = ds_notremor['height'].mean(dim='time')
ds_notremor

## Some verification plots

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 8))
ax2 = ax1.twinx()

ds.sel(station='ALBH').e[0:2500].plot(ax=ax1, color='red')
ds.sel(station='ALBH').tremor_count[0:2500].plot(ax=ax2)

ax1.set_xlabel('day index')
ax1.set_ylabel('e-ref(m)')
ax2.set_ylabel('tremor count')

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 8))
ax2 = ax1.twinx()

ds.sel(station='ALBH', time=slice("2017-03-01", "2017-05-01")).e.plot(ax=ax1, color='red')
ds.sel(station='ALBH', time=slice("2017-03-01", "2017-05-01")).tremor_count.plot(ax=ax2)

ax1.set_xlabel('day index')
ax1.set_ylabel('e-ref(m)')
ax2.set_ylabel('tremor count')

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature



def visualize_graph_torch(ds, tremor_ds):
    fig = plt.figure(figsize=(6,12))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    # ax.set_extent([80, 170, -45, 30], crs=ccrs.PlateCarree())

    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.OCEAN)
    
    ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
        
    plt.scatter(x=ds.longitude, y=ds.latitude, s=30, alpha=1, c='red', zorder=12)

    delta = 0.5
    # add perimeter
    plt.plot([ds.longitude-delta, ds.longitude+delta], [ds.latitude+delta, ds.latitude+delta], c='Blue', linewidth=.5)
    plt.plot([ds.longitude-delta, ds.longitude-delta], [ds.latitude-delta, ds.latitude+delta], c='Blue', linewidth=.5)
    plt.plot([ds.longitude+delta, ds.longitude+delta], [ds.latitude+delta, ds.latitude-delta], c='Blue', linewidth=.5)
    plt.plot([ds.longitude-delta, ds.longitude+delta], [ds.latitude-delta, ds.latitude-delta], c='Blue', linewidth=.5)

        
    plt.scatter(y=tremor_ds.lat.data, x=tremor_ds.lon.data, c=tremor_ds.data, s=2)

    # plt.title(f"{ds.date_start}")
    # plt.xlim((-130, -115))
    # plt.ylim((39, 51))
    plt.show()

In [ ]:
tremor.sel(time=slice("2017-03-01", "2017-05-01")).sum(dim='time').count

In [ ]:
tre_plot = tremor.sel(time=slice("2017-03-01", "2017-05-01")).stack(pos=("lon", "lat")).sum(dim='time')['count']
tre_plot = tre_plot.where(tre_plot>0, drop=True)
visualize_graph_torch(ds.sel(station='ALBH'), tre_plot)

# save dataset

In [ ]:
ds.to_netcdf("../data/clean_daily_CWU.nc")